In [15]:
import transformers
from typing import List

# 全局变量，用于存储分词器，避免重复加载
_tokenizer = None

def get_tokenizer():
    """
    获取或初始化分词器。
    分词器只加载一次，以提高效率。
    """
    global _tokenizer
    if _tokenizer is None:
        # 请确保 './utils/' 路径下有您预训练模型的分词器文件
        # 例如：tokenizer.json, vocab.txt 等
        _tokenizer = transformers.AutoTokenizer.from_pretrained(
            "./utils/",
            trust_remote_code=True,
            use_fast=True  # 强制使用快速分词器
        )
    return _tokenizer

def calculate_token_count(text: str) -> int:
    """
    计算给定字符串的token数量
    
    Args:
        text (str): 要计算token数量的文本
        
    Returns:
        int: 文本的token数量
    """
    tokenizer = get_tokenizer()
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return len(tokens)

# 示例使用
if __name__ == "__main__":
    # 自定义字符串
    my_text = """
      1 SC_val推理：消息缺乏明确权威来源（如巴黎官方或埃菲尔铁塔管理方声明），使用话题标签表明源自社交媒体而非权威新闻。得分：0.15
      2 MC_val推理：无独立消息源佐证（如法国媒体、国际新闻机构），可靠渠道均未支持铁塔亮灯说法。得分：0.1
      3 ES_val推理：视觉证据存在篡改或误用嫌疑，无可靠记录证明该灯光事件，与拉合尔袭击的关联具推测性。得分：0.05
      计算：(0.15+0.1+0.05)/3=0.1
      #### 0.1"""
    
    # 计算token数量
    token_count = calculate_token_count(my_text)
    
    # 输出结果
    print(f"文本: '{my_text}'")
    print(f"Token数量: {token_count}")

文本: '
      1 SC_val推理：消息缺乏明确权威来源（如巴黎官方或埃菲尔铁塔管理方声明），使用话题标签表明源自社交媒体而非权威新闻。得分：0.15
      2 MC_val推理：无独立消息源佐证（如法国媒体、国际新闻机构），可靠渠道均未支持铁塔亮灯说法。得分：0.1
      3 ES_val推理：视觉证据存在篡改或误用嫌疑，无可靠记录证明该灯光事件，与拉合尔袭击的关联具推测性。得分：0.05
      计算：(0.15+0.1+0.05)/3=0.1
      #### 0.1'
Token数量: 147


In [9]:
import transformers
from typing import List

# 全局变量，用于存储分词器，避免重复加载
_tokenizer = None

def get_tokenizer():
    """
    获取或初始化分词器。
    分词器只加载一次，以提高效率。
    """
    global _tokenizer
    if _tokenizer is None:
        # 请确保 './utils/' 路径下有您预训练模型的分词器文件
        # 例如：tokenizer.json, vocab.txt 等
        _tokenizer = transformers.AutoTokenizer.from_pretrained(
            "./utils/",
            trust_remote_code=True,
            use_fast=True  # 强制使用快速分词器
        )
    return _tokenizer

def calculate_and_save_token_counts(
    input_filepath: str,
    output_filepath: str
):
    """
    读取指定文件中的每行文本，计算每行文本的token数量，
    并将结果保存到另一个文件中。

    Args:
        input_filepath (str): 输入文件的路径，每行包含一个文本。
        output_filepath (str): 输出文件的路径，每行保存一个token数量。
    """
    tokenizer = get_tokenizer()
    token_counts = []

    # 读取输入文件
    try:
        with open(input_filepath, 'r', encoding='utf-8') as infile:
            for line in infile:
                # 移除行末的换行符和多余的空白
                text = line.strip()
                if text: # 确保非空行才进行计算
                    tokens = tokenizer.encode(text, add_special_tokens=False)
                    token_counts.append(len(tokens))
                else:
                    token_counts.append(0) # 空行或只有空白的行计为0 token
    except FileNotFoundError:
        print(f"错误: 输入文件 '{input_filepath}' 未找到。请检查文件路径。")
        return
    except Exception as e:
        print(f"读取输入文件时发生错误: {e}")
        return

    # 将token数量保存到输出文件
    try:
        with open(output_filepath, 'w', encoding='utf-8') as outfile:
            for count in token_counts:
                outfile.write(str(count) + '\n')
        print(f"成功将token数量保存到 '{output_filepath}'。共处理了 {len(token_counts)} 行文本。")
    except Exception as e:
        print(f"写入输出文件时发生错误: {e}")

# 示例使用
if __name__ == "__main__":
    # 调用函数处理文件
    calculate_and_save_token_counts(
        input_filepath="./data/twitter_lines.txt",
        output_filepath="./data/twitter_token.txt"
    )

成功将token数量保存到 './data/twitter_token.txt'。共处理了 14355 行文本。


In [13]:
import pandas as pd

def load_data():
    data_rows = []  # 用于存储每一行数据的字典

    try:
        with (
            open("./data/twitter_lines.txt", "r", encoding="utf-8") as text_file,
            open("./data/twitter_label 0 for fake, 1 for true.txt", "r", encoding="utf-8") as label_file, # 修正：添加了逗号
            open("./data/twitter_token.txt", "r", encoding="utf-8") as token_file
        ):
            for text_line, label_line, token_line in zip(text_file, label_file, token_file): # 修正：解包所有三个文件
                text = text_line.strip()
                label = int(label_line.strip())
                token = token_line.strip() # 读取 token 数据

                # 将每行数据存储为字典
                row = {
                    'text': text,
                    'label': label,
                    'token': token
                }
                data_rows.append(row)

    except FileNotFoundError as e:
        pass
    # 从字典列表创建DataFrame
    df = pd.DataFrame(data_rows)
    return df

df = load_data()
df

,text,label,token
0,chan spots a Boston Marathon bombings suspect,0,8
1,So THIS is the face of the BostonMarathon bomb...,1,21
2,dazonic U said it Appears that a Reddit user f...,1,22
3,Os caras do chan identificaram o terrorista da...,0,19
4,RT YourAnonNews New FBI BostonMarathon Suspect...,1,13
...,...,...,...
14350,rapo Wow DylanByers RT DailyIntel The cover of...,1,19
14351,newyork hurricane sandy,0,5
14352,Tomb of the Unknown Soldier during Hurricane S...,0,13
14353,NY The City and the Storm The Look of PostSand...,1,14


In [ ]:
import pandas as pd
import numpy as np

def load_data():
    data_rows = []  # 用于存储每一行数据的字典
    try:
        with (
            open("./data/twitter_lines.txt", "r", encoding="utf-8") as text_file,
            open("./data/twitter_label 0 for fake, 1 for true.txt", "r", encoding="utf-8") as label_file,
            open("./data/twitter_token.txt", "r", encoding="utf-8") as token_file
        ):
            for text_line, label_line, token_line in zip(text_file, label_file, token_file):
                text = text_line.strip()
                label = int(label_line.strip())
                token_count = int(token_line.strip())  # 直接读取token数量
                
                row = {
                    'text': text,
                    'label': label,
                    'token_count': token_count  # 直接存储token数量
                }
                data_rows.append(row)
    except FileNotFoundError as e:
        print(f"文件未找到: {e}")
    
    df = pd.DataFrame(data_rows)
    return df

def sample_by_token_count(df, n=200):
    # 只保留 token_count <= 100 的行（效果等同于删除 >100 的行）
    df1 = df[df['token_count'] <= 100]
    df2 = df1[df1['token_count'] > 3]
    # 1. 计算label比例
    label_counts = df2['label'].value_counts(normalize=True)
    
    # 2. 按label分组
    df_0 = df2[df2['label'] == 0].copy()
    df_1 = df2[df2['label'] == 1].copy()
    
    # 3. 计算每组应该抽取的样本数（保持原始比例）
    n_0 = int(n * label_counts[0])
    n_1 = n - n_0  # 确保总数是200
    
    # 4. 对每组按token_count排序并均匀抽样
    def stratified_sample(group, size):
        if len(group) == 0:
            return group
        # 先按token_count排序
        group = group.sort_values('token_count')
        # 计算间隔以确保均匀分布
        step = max(1, len(group) // size)
        indices = np.linspace(0, len(group)-1, size, dtype=int)
        return group.iloc[indices]
    
    sample_0 = stratified_sample(df_0, n_0)
    sample_1 = stratified_sample(df_1, n_1)
    
    # 5. 合并样本
    result = pd.concat([sample_0, sample_1]).sample(frac=1, random_state=42).reset_index(drop=True)
    
    return result

# 加载数据
df = load_data()

# 抽样200行
sampled_df = sample_by_token_count(df, 200)

# 按token_count升序排序
sampled_df.sort_values("token_count", ascending=True, inplace=True)

# 显示结果
print(sampled_df)
print("\nLabel分布:")
print(sampled_df['label'].value_counts(normalize=True))
print("\nToken数量统计:")
print(sampled_df['token_count'].describe())

                                                  text  label  token_count
121                               PrayForAmerica Sandy      1            4
136                                 hurricane sandy NY      0            4
180                                     HurricaneSandy      0            4
104                             This Morning NYC Sandy      1            4
111                                      Sandy at work      0            4
..                                                 ...    ...          ...
112  RT BREAKlNG WANTED  yrold Dzhokhar TsarnaevSus...      1           31
137  Hoy os estn colando multitud de fotomontajes d...      0           31
195  aparmisen No s si esta foto del temporal en NY...      0           33
166  RT odtumuhalefet Gericilikten katillerden kork...      0           45
191  Fuji created huge lenticular clouds and they w...      1           54

[200 rows x 3 columns]

Label分布:
label
0    0.54
1    0.46
Name: proportion, dtype: float64

Token数

In [12]:
import transformers
import os
import re
from typing import List

# 全局分词器
_tokenizer = None

def get_tokenizer():
    global _tokenizer
    if _tokenizer is None:
        _tokenizer = transformers.AutoTokenizer.from_pretrained(
            "./utils/",
            trust_remote_code=True,
            use_fast=True
        )
    return _tokenizer

def clean_weibo_text(text: str) -> str:
    """
    微博文本专用清洗：
    1. 保留中文、英文、数字
    2. 移除URL、@提及、话题标签
    3. 移除所有标点符号（包括中文标点）
    4. 合并连续空白
    """
    # 移除URL和@提及
    text = re.sub(r'http\S+|@\S+|#\S+', '', text)
    
    # 只保留中文、英文、数字和空格
    # 中文Unicode范围：\u4e00-\u9fa5
    # 英文数字：a-zA-Z0-9
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9\s]', '', text)
    
    # 合并连续空白并去除首尾空白
    text = ' '.join(text.split()).strip()
    
    return text

def process_weibo_file(input_filepath: str, output_filepath: str):
    tokenizer = get_tokenizer()
    token_counts = []

    try:
        with open(input_filepath, 'r', encoding='utf-8') as infile:
            lines = infile.readlines()
            
            for i in range(2, len(lines), 3):  # 每3行取第3行
                text = lines[i].strip()
                text = clean_weibo_text(text)
                
                if text:
                    tokens = tokenizer.encode(text, add_special_tokens=False)
                    token_counts.append(len(tokens))
                else:
                    token_counts.append(0)

    except Exception as e:
        print(f"处理文件 {input_filepath} 时出错: {e}")
        return False

    try:
        with open(output_filepath, 'w', encoding='utf-8') as outfile:
            outfile.write('\n'.join(map(str, token_counts)))
        print(f"处理完成: {input_filepath} → {output_filepath} ({len(token_counts)}条)")
        return True
    except Exception as e:
        print(f"写入 {output_filepath} 时出错: {e}")
        return False

def process_weibo_dataset(input_dir: str, output_dir: str):
    """处理微博数据集"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    processed = 0
    for filename in os.listdir(input_dir):
        if filename.endswith('.txt') and not filename.startswith('token_count_'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"token_count_{filename}")
            
            if process_weibo_file(input_path, output_path):
                processed += 1

    print(f"\n处理完成！共处理 {processed} 个文件")

if __name__ == "__main__":
    # 示例使用
    process_weibo_dataset(
        input_dir="./data/tweets/",
        output_dir="./data/tweets/"
    )

处理完成: ./data/tweets/test_nonrumor.txt → ./data/tweets/token_count_test_nonrumor.txt (996条)
处理完成: ./data/tweets/test_rumor.txt → ./data/tweets/token_count_test_rumor.txt (1000条)
处理完成: ./data/tweets/train_nonrumor.txt → ./data/tweets/token_count_train_nonrumor.txt (3783条)
处理完成: ./data/tweets/train_rumor.txt → ./data/tweets/token_count_train_rumor.txt (3748条)

处理完成！共处理 4 个文件


In [15]:
import pandas as pd
import numpy as np
import os
import re

def clean_weibo_text(text: str) -> str:
    """
    微博文本专用清洗：
    1. 保留中文、英文、数字
    2. 移除URL、@提及、话题标签
    3. 移除所有标点符号（包括中文标点）
    4. 合并连续空白
    """
    if not isinstance(text, str):
        return ""
    
    # 移除URL和@提及
    text = re.sub(r'http\S+|@\S+|#\S+', '', text)
    
    # 只保留中文、英文、数字和空格
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9\s]', '', text)
    
    # 合并连续空白并去除首尾空白
    text = ' '.join(text.split()).strip()
    
    return text

def load_tweets_data():
    data_rows = []  # 用于存储所有数据的列表
    
    # 定义要处理的文件列表及其对应标签
    tweet_files = [
        ('test_nonrumor.txt', 1),
        ('test_rumor.txt', 0),
        ('train_nonrumor.txt', 1),
        ('train_rumor.txt', 0)
    ]
    
    # 处理每个文件
    for filename, label in tweet_files:
        text_filepath = os.path.join('./data/tweets/', filename)
        token_filepath = os.path.join('./data/tweets/', f'token_count_{filename}')
        
        try:
            # 读取文本文件（每3行取第3行）并进行清洗
            with open(text_filepath, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                texts = [clean_weibo_text(lines[i].strip()) for i in range(2, len(lines), 3)]
            
            # 读取对应的token计数文件
            with open(token_filepath, 'r', encoding='utf-8') as f:
                tokens = [int(line.strip()) for line in f.readlines()]
            
            # 确保文本和token数量匹配
            min_len = min(len(texts), len(tokens))
            for i in range(min_len):
                data_rows.append({
                    'label': label,
                    'token_count': tokens[i],
                    'text': texts[i],          # 存储清洗后的文本
                    'original_text': lines[2+i*3].strip() if i*3+2 < len(lines) else ""  # 可选：存储原始文本
                })
                
        except FileNotFoundError:
            print(f"文件未找到: {text_filepath} 或 {token_filepath}")
            continue
        except Exception as e:
            print(f"处理文件 {text_filepath} 时出错: {e}")
            continue
    
    # 创建DataFrame
    df = pd.DataFrame(data_rows)
    return df

def sample_by_token_count(df, n=200):
    # 只保留 token_count > 30 的行
    df_filtered = df[df['token_count'] > 30].copy()
    
    # 1. 计算label比例
    label_counts = df_filtered['label'].value_counts(normalize=True)
    
    # 2. 按label分组
    df_0 = df_filtered[df_filtered['label'] == 0].copy()
    df_1 = df_filtered[df_filtered['label'] == 1].copy()
    
    # 3. 计算每组应该抽取的样本数（保持原始比例）
    n_0 = int(n * label_counts[0])
    n_1 = n - n_0  # 确保总数是n
    
    # 4. 对每组按token_count排序并均匀抽样
    def stratified_sample(group, size):
        if len(group) == 0:
            return group
        group = group.sort_values('token_count')
        indices = np.linspace(0, len(group)-1, size, dtype=int)
        return group.iloc[indices]
    
    sample_0 = stratified_sample(df_0, n_0)
    sample_1 = stratified_sample(df_1, n_1)
    
    # 5. 合并样本
    result = pd.concat([sample_0, sample_1]).reset_index(drop=True)
    
    return result

# 主程序
if __name__ == "__main__":
    # 加载并清洗数据
    df = load_tweets_data()
    
    # 抽样200行
    sampled_df = sample_by_token_count(df, 200)
    
    # 按token_count升序排序
    sampled_df.sort_values("token_count", ascending=True, inplace=True)
    
    # 显示结果
    print("抽样结果（前5行）：")
    print(sampled_df[['label', 'token_count', 'text']].head())
    
    print("\nLabel分布:")
    print(sampled_df['label'].value_counts(normalize=True))
    
    print("\nToken数量统计:")
    print(sampled_df['token_count'].describe())
    
    # 可选：保存清洗后的数据
    sampled_df.to_csv('./data/cleaned_sampled_data.csv', index=False, encoding='utf-8-sig')

抽样结果（前5行）：
     label  token_count                                               text
0        0           31  请国家还原真相nbspnbsp因为相关部门的过失害死了这么多人nbspnbspnbsp现在连...
102      1           31  Stark家族恭喜权力的游戏在第67届艾美奖中获得24项提名与12座奖杯私生子囧斯诺则表示我...
103      1           32  今天上午山东平邑一石膏矿垮塌引发震动目前有多人被埋伤亡不明据 正式测定振动相当于4级地震据新...
1        0           32  转发一位叫徐敬的女孩21岁请速回雅安水城县人民医院妈妈伤的很严重想见她最后一面爸爸号码151...
2        0           33  男主已被乱棍打死好么 来自网易新闻广西南宁一奔驰车闯拆迁现场撞人 致3人受伤更多精彩尽在 O...

Label分布:
label
0    0.51
1    0.49
Name: proportion, dtype: float64

Token数量统计:
count    200.000000
mean      69.610000
std       16.295571
min       31.000000
25%       61.000000
50%       72.000000
75%       80.000000
max      125.000000
Name: token_count, dtype: float64


In [4]:
import pandas as pd
import numpy as np
import os
from scipy.stats import chi2_contingency
import time
from collections import defaultdict

# 配置参数
PROMPTS = ["cod", "cot","baseline", "basebaseline"]  # 使用的提示策略列表
DATASETS = ["twitter","weibo"]                         # 要分析的数据集列表
# MODEL = "deepseek-v3"                          # 使用的模型名称
MODEL = "qwen2.5-32b"                          # 使用的模型名称
SHOT = 2                                       # few-shot数量
OUTPUT_DIR = "./analysis/"                     # 输出目录
os.makedirs(OUTPUT_DIR, exist_ok=True)         # 确保输出目录存在

def calculate_phi(confusion_matrix):
    """计算两个二分类变量的Phi相关系数
    参数:
        confusion_matrix: 混淆矩阵(列联表)
    返回:
        phi系数(范围[-1,1])
    """
    # 计算卡方值
    chi2, _, _, _ = chi2_contingency(confusion_matrix, correction=False)
    n = confusion_matrix.sum().sum()  # 总样本数
    # 计算phi系数并保留方向性
    return np.sqrt(chi2 / n) * np.sign(confusion_matrix.iloc[0,0] * confusion_matrix.iloc[1,1] - 
                                      confusion_matrix.iloc[0,1] * confusion_matrix.iloc[1,0])

def load_predictions(dataset, prompt):
    """加载预测数据及其所有可用字段
    参数:
        dataset: 数据集名称
        prompt: 提示策略名称
    返回:
        包含所有预测数据的DataFrame
    """
    file_path = f"./results/{dataset}-{MODEL}-{prompt}-{SHOT}.csv"
    try:
        df = pd.read_csv(file_path)
        # 确保必须的列存在
        if 'predict' not in df.columns or 'gt' not in df.columns:
            raise ValueError("CSV文件缺少必要列(predict/gt)")
        
        # 添加衍生字段
        df['is_correct'] = df['predict'] == df['gt']  # 预测是否正确
        
        # 处理时间字段(如果存在)
        if 'time' in df.columns:
            df['time'] = pd.to_numeric(df['time'], errors='coerce')  # 转换为数值
            
        # 处理输入token数(如果存在)
        if 'qs_token' in df.columns:
            df['qs_token'] = pd.to_numeric(df['qs_token'], errors='coerce')
            
        # 处理输出token数(如果存在)
        if 'out_token' in df.columns:
            df['out_token'] = pd.to_numeric(df['out_token'], errors='coerce')
            
        return df
    except Exception as e:
        print(f"加载{file_path}出错: {str(e)}")
        return None

def calculate_pairwise_metrics(predictions_dict):
    """计算所有提示策略两两之间的指标
    参数:
        predictions_dict: 包含各提示策略预测结果的字典
    返回:
        agreement_matrix: 一致性矩阵(百分比)
        phi_matrix: phi系数矩阵(百分比)
    """
    prompts = list(predictions_dict.keys())  # 获取所有提示策略名称
    # 初始化结果矩阵
    agreement_matrix = pd.DataFrame(index=prompts, columns=prompts, dtype=float)
    phi_matrix = pd.DataFrame(index=prompts, columns=prompts, dtype=float)
    
    # 计算两两指标
    for p1 in prompts:
        for p2 in prompts:
            df1 = predictions_dict[p1]
            df2 = predictions_dict[p2]
            
            # 计算一致性百分比(预测结果相同的比例)
            agreement = (df1['predict'] == df2['predict']).mean() * 100
            agreement_matrix.loc[p1, p2] = agreement
            
            # 计算phi系数
            confusion = pd.crosstab(df1['predict'], df2['predict'])  # 构建列联表
            if confusion.shape == (2, 2):  # 确保是2x2矩阵
                phi = calculate_phi(confusion) * 100
            else:  # 如果某一类全为0/1时的处理
                phi = 100 if (df1['predict'] == df2['predict']).all() else 0
            phi_matrix.loc[p1, p2] = phi
    
    return agreement_matrix, phi_matrix

def calculate_detailed_stats(predictions_dict):
    """为每个提示策略计算详细的统计信息（优化版）
    参数:
        predictions_dict: 包含各提示策略预测结果的字典
    返回:
        包含详细统计信息的DataFrame
    
    修改说明:
        1. 将混淆矩阵分量改为百分比形式
        2. 时间统计只保留平均值和标准差
        3. token统计只保留输出token的平均值和标准差
    """
    stats = []  # 存储各提示策略的统计信息
    
    for prompt, df in predictions_dict.items():
        if df is None:  # 跳过加载失败的情况
            continue
        
        total_samples = len(df)
        
        # 基础统计信息
        stat = {
            'prompt': prompt,  # 提示策略名称
            'accuracy': df['is_correct'].mean() * 100,  # 准确率(百分比)
            'num_samples': total_samples,  # 样本数量
            
            # 混淆矩阵各分量(改为百分比形式)
            'true_positives_ratio': ((df['predict'] == 1) & (df['gt'] == 1)).sum() / total_samples * 100,
            'false_positives_ratio': ((df['predict'] == 1) & (df['gt'] == 0)).sum() / total_samples * 100,
            'true_negatives_ratio': ((df['predict'] == 0) & (df['gt'] == 0)).sum() / total_samples * 100,
            'false_negatives_ratio': ((df['predict'] == 0) & (df['gt'] == 1)).sum() / total_samples * 100,
            
            # 保留原始计数字段(如果需要)
            'true_positives': ((df['predict'] == 1) & (df['gt'] == 1)).sum(),
            'false_positives': ((df['predict'] == 1) & (df['gt'] == 0)).sum(),
            'true_negatives': ((df['predict'] == 0) & (df['gt'] == 0)).sum(),
            'false_negatives': ((df['predict'] == 0) & (df['gt'] == 1)).sum(),
        }
        
        # 时间统计(只保留平均值和标准差)
        if 'time' in df.columns:
            stat.update({
                'avg_time': df['time'].mean(),  # 平均时间
                'time_std': df['time'].std()    # 时间标准差
            })
        
        # Token统计(只保留输出token的平均值和标准差)
        if 'out_token' in df.columns:
            stat.update({
                'avg_output_tokens': df['out_token'].mean(),  # 平均输出token数
                'std_output_tokens': df['out_token'].std()    # 输出token标准差
            })
        
        stats.append(stat)
    
    return pd.DataFrame(stats)

def analyze_dataset(dataset):
    """对单个数据集进行综合分析
    参数:
        dataset: 数据集名称
    返回:
        该数据集的详细统计信息
    """
    print(f"\n正在分析数据集: {dataset}")
    start_time = time.time()  # 记录开始时间
    
    # 加载所有提示策略的预测结果
    predictions = {prompt: load_predictions(dataset, prompt) for prompt in PROMPTS}
    
    # 计算各项指标
    agreement_matrix, phi_matrix = calculate_pairwise_metrics(predictions)  # 一致性矩阵和phi矩阵
    detailed_stats = calculate_detailed_stats(predictions)  # 详细统计信息
    
    # 保存结果
    save_results(dataset, {
        'agreement_matrix': agreement_matrix,
        'phi_matrix': phi_matrix,
        'detailed_stats': detailed_stats
    })
    
    # 打印耗时信息
    print(f"完成{dataset}分析，耗时{time.time()-start_time:.2f}秒")
    return detailed_stats

def save_results(dataset, results):
    """保存数据集的分析结果到文件
    参数:
        dataset: 数据集名称
        results: 包含各种分析结果的字典
    """
    prefix = f"{dataset}_{MODEL}"  # 文件名前缀
    
    # 保存矩阵数据(一致性和phi矩阵)
    for name, matrix in [('agreement', results['agreement_matrix']), 
                        ('phi', results['phi_matrix'])]:
        if matrix is not None:
            df = matrix.reset_index().rename(columns={'index': 'prompt'})
            df['dataset'] = dataset  # 添加数据集列
            df.to_csv(os.path.join(OUTPUT_DIR, f"{prefix}_{name}.csv"), 
                     index=False, float_format="%.2f")  # 保留两位小数
    
    # 保存详细统计数据
    if results['detailed_stats'] is not None:
        results['detailed_stats'].to_csv(
            os.path.join(OUTPUT_DIR, f"{prefix}_stats.csv"),
            index=False, float_format="%.2f"  # 保留两位小数
        )

def analyze_all_datasets():
    """主分析函数，处理所有数据集"""
    all_stats = []  # 存储所有数据集的统计信息
    
    for dataset in DATASETS:
        stats = analyze_dataset(dataset)  # 分析当前数据集
        if stats is not None:
            stats['dataset'] = dataset  # 添加数据集列
            all_stats.append(stats)  # 添加到总统计列表
    
    # 合并并保存所有数据集的统计信息
    if all_stats:
        combined_stats = pd.concat(all_stats)
        combined_stats.to_csv(
            os.path.join(OUTPUT_DIR, f"{MODEL}_combined_stats.csv"),
            index=False, float_format="%.2f"  # 保留两位小数
        )
    
    print("\n分析完成! 结果已保存至:", OUTPUT_DIR)

if __name__ == "__main__":
    analyze_all_datasets()  # 执行分析


正在分析数据集: twitter
完成twitter分析，耗时0.36秒

正在分析数据集: weibo
完成weibo分析，耗时0.31秒

分析完成! 结果已保存至: ./analysis/


In [1]:
import pandas as pd
import numpy as np
import os

# 配置参数
PROMPTS = ["cod", "baseline", "basebaseline"]
DATASETS = ["twitter"]
MODEL = "qwen2.5-32b"
SHOT = 2
OUTPUT_DIR = "./analysis/"

def analyze_datasets():
    # 初始化一个DataFrame来存储每个数据集的合并正确率
    merged_accuracy_results = pd.DataFrame(columns=['Dataset', 'Merged_Accuracy'])

    for dataset in DATASETS:
        print(f"\n正在处理数据集: {dataset}")

        all_predictions = []
        gt_labels = None

        # 读取所有提示策略的预测结果
        for prompt in PROMPTS:
            file_path = f"./results/{dataset}-{MODEL}-{prompt}-{SHOT}.csv"
            try:
                df = pd.read_csv(file_path)
                all_predictions.append(df['predict'].astype(int))
                if gt_labels is None:
                    gt_labels = df['gt'].astype(int)
            except Exception as e:
                print(f"读取{dataset}-{prompt}失败: {str(e)}")
                continue

        if not all_predictions:
            print(f"没有为数据集 {dataset} 找到有效的预测结果。")
            continue

        # 将所有预测结果合并成一个DataFrame
        predictions_df = pd.DataFrame(all_predictions).T
        predictions_df.columns = PROMPTS

        # 使用投票法进行合并：选择出现次数最多的预测
        # 如果有平局，这里默认选择第一个遇到的类别（例如0或1）
        merged_predictions = predictions_df.mode(axis=1)[0]

        # 计算合并后的正确率
        if gt_labels is not None:
            correct_predictions = (merged_predictions == gt_labels).sum()
            total_predictions = len(gt_labels)
            merged_accuracy = (correct_predictions / total_predictions) * 100
            print(f"数据集 {dataset} 的合并正确率: {merged_accuracy:.2f}%")

            # 将结果添加到汇总DataFrame
            merged_accuracy_results.loc[len(merged_accuracy_results)] = [dataset, merged_accuracy]
        else:
            print(f"数据集 {dataset} 缺少真实标签 (gt)。")

    # 保存最终合并正确率的表格
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    output_filename = os.path.join(OUTPUT_DIR, f"{MODEL}_merged_accuracy.csv")
    merged_accuracy_results.to_csv(output_filename, index=False, float_format="%.2f")
    print("\n分析完成！合并正确率结果已保存至:", output_filename)

if __name__ == "__main__":
    analyze_datasets()


正在处理数据集: twitter
数据集 twitter 的合并正确率: 63.52%

分析完成！合并正确率结果已保存至: ./analysis/qwen2.5-32b_merged_accuracy.csv
